<h1>Words Size Regression - Random Forest</h1>
<br>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! mkdir ascii
! tar xzf drive/MyDrive/ocr/database/ascii.tgz -C ascii
! mkdir lines
! tar xzf drive/MyDrive/ocr/database/lines.tgz -C lines
! mkdir words
! tar xzf drive/MyDrive/ocr/database/words.tgz -C words
! cp drive/MyDrive/ocr/src/importation.py importation.py

Mounted at /content/drive


In [5]:
from importation import Importation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


# Importation des données - preprocessing

In [8]:
imp = Importation('database')
df = imp.get_words().sample(frac=1, random_state=1).reset_index()
# n_lettres_max=11
# df = df[df.word_size<=n_lettres_max][['file', 'word_size', 'label']]
df.word_size = df.word_size.astype('float32')
print("Nombre d'enregistrements ", df.shape[0])


df = df[['file', 'word_size', 'label']]
df.info()
image_width = 128
image_height = 32
batch_size = 64
img_size = (image_width, image_height)
df.head(20)


X = []
Y = []

dim = (image_width, image_height)
for file, word_size in zip(df["file"],df["word_size"]):
    image = cv2.resize(cv2.imread(file, cv2.IMREAD_GRAYSCALE),
                  dim, cv2.IMREAD_GRAYSCALE)
    X.append(np.array(image))
    Y.append(word_size)

# Normalise input data to range [0,1]
X = np.array(X)
X = X.astype('float32')
shape = list(X.shape)
shape.append(1)
X = X.reshape(shape)
X /= 255.0
X = X.reshape([-1, image_width*image_height])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

Nombre d'enregistrements  96454
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96454 entries, 0 to 96453
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   file       96454 non-null  object 
 1   word_size  96454 non-null  float32
 2   label      96454 non-null  object 
dtypes: float32(1), object(2)
memory usage: 1.8+ MB


# Random Forest

In [9]:
# Random Forest
rf_clf = RandomForestRegressor(n_estimators = 90, criterion='squared_error', max_depth = 5, n_jobs = -1)
rf_clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_estimators=90, n_jobs=-1)

# Prédiction

In [11]:
# prediction
test_pred = rf_clf.predict(X_test)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, test_pred)
print("Mean abolute error : ", mae)

Mean abolute error :  1.3679947377414092
